In [1]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from Parser import parse_frames
from scipy.signal import medfilt
from Functions import analyse_each_rep, find_extremas, filter_extremas, numpy_fillna, DTWDistance
from JointAngles import JointAngles
import numpy as np
import glob
import os
import pickle
import pandas as pd
from functools import reduce


In [3]:
# input_folder = "C:\\Users\\altaok\\Documents\\GitHub\\IndividualProject\\keypoints_for_all\\shoulder press"
input_folder = 'C:\\Users\\ak5u16\\Desktop\\IndividualProject\\keypoints_for_all\\shoulder press'
folder_paths = glob.glob(os.path.join(input_folder, 'shoulder_press*'))
points_folder_name = os.path.basename(input_folder)

print(len(folder_paths))


def get_data_for_dataset(folder_paths, points_folder_name):
    angle_arrays = []
    for folder in folder_paths:
        video_name = os.path.basename(folder)
        label = 0 if '_correct' in folder else 1
        frame_poses = parse_frames(folder)
        joint_angles = JointAngles(points_folder_name, frame_poses)
        
        left_upArm_forearm_angles = np.array(joint_angles.left_upArm_forearm_angles)
        left_upArm_forearm_angles = np.nan_to_num(left_upArm_forearm_angles)
        left_upArm_forearm_angles_filtered = medfilt(medfilt(left_upArm_forearm_angles, 5), 5)
            
        right_upArm_forearm_angles = np.array(joint_angles.right_upArm_forearm_angles)
        right_upArm_forearm_angles = np.nan_to_num(right_upArm_forearm_angles)
        right_upArm_forearm_angles_filtered = medfilt(medfilt(right_upArm_forearm_angles, 5), 5)
            
        left_upArm_trunk_angles = np.array(joint_angles.left_upArm_trunk_angles)
        left_upArm_trunk_angles = np.nan_to_num(left_upArm_trunk_angles)
        left_upArm_trunk_angles_filtered = medfilt(medfilt(left_upArm_trunk_angles, 5), 5)
            
        right_upArm_trunk_angles = np.array(joint_angles.right_upArm_trunk_angles)
        right_upArm_trunk_angles = np.nan_to_num(right_upArm_trunk_angles)
        right_upArm_trunk_angles_filtered = medfilt(medfilt(right_upArm_trunk_angles, 5), 5)
            
        extremas1 = filter_extremas(left_upArm_trunk_angles_filtered, find_extremas(left_upArm_trunk_angles_filtered, maxima=False), maxima=False)
        extremas2 = filter_extremas(right_upArm_trunk_angles_filtered, find_extremas(right_upArm_trunk_angles_filtered, maxima=False), maxima=False)
            
        angle_arrays.append((label, extremas1, extremas2, [left_upArm_forearm_angles, right_upArm_forearm_angles, left_upArm_trunk_angles, right_upArm_trunk_angles]))


    return angle_arrays
                                             
                                    

def fill_dataframe(angle_arrays, exercise_folder_name):
    df_list = []
    
    print('Filling dataset with ' + str(exercise_folder_name) + ' data...')
   
    for tup in angle_arrays:
        label = tup[0]
        extremas1 = tup[1]
        extremas2 = tup[2]
        uf_angles1, uf_angles2, ut_angles1, ut_angles2 = tup[3]
        # Extract rep angles
        each_rep_angles = analyse_each_rep('shoulder press', 'dataset', extremas1=extremas1, uf_angles1=uf_angles1, ut_angles1=ut_angles1, extremas2=extremas2, uf_angles2=uf_angles2, ut_angles2=ut_angles2)
        #print(len(each_rep_angles))
        s1 = pd.Series(each_rep_angles, name='Angle_array')
        s2 = pd.Series([label for n in range(len(each_rep_angles))], name='Label')
        df = pd.concat([s1,s2], axis=1)
        df_list.append(df)

    return pd.concat(df_list).reset_index(drop=True)
        

56


In [11]:
angle_arrays = get_data_for_dataset(folder_paths, points_folder_name) 
df = fill_dataframe(angle_arrays, points_folder_name)
print(df.info())

Filling dataset with shoulder press data...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1688 entries, 0 to 1687
Data columns (total 2 columns):
Angle_array    1688 non-null object
Label          1688 non-null int64
dtypes: int64(1), object(1)
memory usage: 26.5+ KB
None


In [21]:
from sklearn.model_selection import train_test_split
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

y = df['Label']
x = df['Angle_array']
#x = numpy_fillna(x)


In [26]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
X_train.shape

(1350,)

In [27]:
X_test.shape

(338,)

In [30]:
k_range = range(1, 26) #26
scores = {}
scores_list = []
for k in k_range:
    knn_clf = KNeighborsTimeSeriesClassifier(n_neighbors=k, metric="dtw", n_jobs=-1)
    #knn_clf = KNeighborsClassifier(k, n_jobs=-1)
    knn_clf.fit(numpy_fillna(X_train.values), y_train)
    #knn_clf.fit(X_train, y_train)
    predicted_labels = knn_clf.predict(numpy_fillna(X_test.values))
    acc = metrics.accuracy_score(y_test, predicted_labels)
    scores[k] = acc
    #print("Correct classification rate:", acc)
    #print('\n')
    #print(metrics.classification_report(y_test, predicted_labels))
    #print('F1 score: ' + str(metrics.f1_score(y_test, predicted_labels, average='macro')))


for key, value in scores.items():   
    if value == max(scores.values()):
        print('best value for k: ' + str(key) + ' - ' + str(value))
        break

best value for k: 3 - 0.9497041420118343


In [29]:
with open('datasets/shoulder_press_df.pickle', 'wb') as f:
    pickle.dump(df, f)